<a href="https://colab.research.google.com/github/rkrissada/100DayOfMLCode/blob/master/mlapis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Using Machine Learning APIs </h1>

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Copy-paste your API Key here:

In [0]:
APIKEY="API Key"  # Replace with your API key

<b> Note: Make sure you generate an API Key and replace the value above. The sample key will not work.</b>

From the same API console, choose "Dashboard" on the left-hand menu and "Enable API".

Enable the following APIs for your project (search for them) if they are not already enabled:
<ol>
<li> Google Translate API </li>
<li> Google Cloud Vision API </li>
<li> Google Natural Language API </li>
<li> Google Cloud Speech API </li>
</ol>

Finally, because we are calling the APIs from Python (clients in many other languages are available), let's install the Python package (it's not installed by default on Datalab)

In [0]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /usr/local/envs/py3env/lib/python3.5/site-packages (1.7.8)


<h2> Invoke Translate API </h2>

In [0]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# use the service
inputs = ['is it really this easy?', 'amazing technology', 'wow']
outputs = service.translations().list(source='en', target='fr', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print("{0} -> {1}".format(input, output['translatedText']))

is it really this easy? -> est-ce vraiment si facile?
amazing technology -> technologie incroyable
wow -> sensationnel


<h2> Invoke Vision API </h2>

The Vision API can work off an image in Cloud Storage or embedded directly into a POST message. I'll use Cloud Storage and do OCR on this image: <img src="https://storage.googleapis.com/cloud-training-demos/vision/sign2.jpg" width="200" />.  That photograph is from http://www.publicdomainpictures.net/view-image.php?image=15842


In [0]:
# Running Vision API
import base64
IMAGE="gs://cloud-training-demos/vision/sign2.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses)

{'responses': [{'textAnnotations': [{'boundingPoly': {'vertices': [{'y': 104, 'x': 152}, {'y': 104, 'x': 1082}, {'y': 658, 'x': 1082}, {'y': 658, 'x': 152}]}, 'locale': 'zh', 'description': '请您爱护和保\n护卫生创建优\n美水环境\n'}, {'boundingPoly': {'vertices': [{'y': 104, 'x': 179}, {'y': 104, 'x': 321}, {'y': 243, 'x': 321}, {'y': 243, 'x': 179}]}, 'description': '请'}, {'boundingPoly': {'vertices': [{'y': 108, 'x': 329}, {'y': 108, 'x': 471}, {'y': 243, 'x': 471}, {'y': 243, 'x': 329}]}, 'description': '您'}, {'boundingPoly': {'vertices': [{'y': 108, 'x': 479}, {'y': 108, 'x': 775}, {'y': 243, 'x': 775}, {'y': 243, 'x': 479}]}, 'description': '爱护'}, {'boundingPoly': {'vertices': [{'y': 111, 'x': 786}, {'y': 111, 'x': 925}, {'y': 243, 'x': 925}, {'y': 243, 'x': 786}]}, 'description': '和'}, {'boundingPoly': {'vertices': [{'y': 104, 'x': 933}, {'y': 104, 'x': 1079}, {'y': 239, 'x': 1079}, {'y': 239, 'x': 933}]}, 'description': '保'}, {'boundingPoly': {'vertices': [{'y': 300, 'x': 165}, {'y': 300, 'x': 3

In [0]:
foreigntext = responses['responses'][0]['textAnnotations'][0]['description']
foreignlang = responses['responses'][0]['textAnnotations'][0]['locale']
print(foreignlang, foreigntext)

zh 请您爱护和保
护卫生创建优
美水环境



<h2> Translate sign </h2>

In [0]:
inputs=[foreigntext]
outputs = service.translations().list(source=foreignlang, target='en', q=inputs).execute()
# print(outputs)
for input, output in zip(inputs, outputs['translations']):
  print("{0} -> {1}".format(input, output['translatedText']))

请您爱护和保
护卫生创建优
美水环境
 -> Please protect and protect the health to create a beautiful water environment


<h2> Sentiment analysis with Language API </h2>

Let's evaluate the sentiment of some famous quotes using Google Cloud Natural Language API.

In [0]:
lservice = build('language', 'v1beta1', developerKey=APIKEY)
quotes = [
  'To succeed, you must have tremendous perseverance, tremendous will.',
  'It’s not that I’m so smart, it’s just that I stay with problems longer.',
  'Love is quivering happiness.',
  'Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.',
  'What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?',
  'When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. '
]
for quote in quotes:
  response = lservice.documents().analyzeSentiment(
    body={
      'document': {
         'type': 'PLAIN_TEXT',
         'content': quote
      }
    }).execute()
  polarity = response['documentSentiment']['polarity']
  magnitude = response['documentSentiment']['magnitude']
  print('POLARITY=%s MAGNITUDE=%s for %s' % (polarity, magnitude, quote))

POLARITY=1 MAGNITUDE=0.9 for To succeed, you must have tremendous perseverance, tremendous will.
POLARITY=-1 MAGNITUDE=0.5 for It’s not that I’m so smart, it’s just that I stay with problems longer.
POLARITY=1 MAGNITUDE=0.9 for Love is quivering happiness.
POLARITY=1 MAGNITUDE=0.9 for Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.
POLARITY=1 MAGNITUDE=0.2 for What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?
POLARITY=-1 MAGNITUDE=0.4 for When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. 


<h2> Speech API </h2>

The Speech API can work on streaming data, audio content encoded and embedded directly into the POST message, or on a file on Cloud Storage. Here I'll pass in this <a href="https://storage.googleapis.com/cloud-training-demos/vision/audio.raw">audio file</a> in Cloud Storage.

In [0]:
sservice = build('speech', 'v1', developerKey=APIKEY)
response = sservice.speech().recognize(
    body={
        'config': {
            'languageCode' : 'en-US',
            'encoding': 'LINEAR16',
            'sampleRateHertz': 16000
        },
        'audio': {
            'uri': 'gs://cloud-training-demos/vision/audio.raw'
            }
        }).execute()
print(response)

{'results': [{'alternatives': [{'confidence': 0.9835046, 'transcript': 'how old is the Brooklyn Bridge'}]}]}


In [0]:
print(response['results'][0]['alternatives'][0]['transcript'])
print('Confidence=%f' % response['results'][0]['alternatives'][0]['confidence'])

how old is the Brooklyn Bridge
Confidence=0.983505


<h2> Clean up </h2>

Remember to delete the API key by visiting <a href="http://console.cloud.google.com/apis">API console</a>.

If necessary, commit all your notebooks to git.

If you are running Datalab on a Compute Engine VM or delegating to one, remember to stop or shut it down so that you are not charged.


## Challenge Exercise

Here are a few portraits from the Metropolitan Museum of Art, New York (they are part of a [BigQuery public dataset](https://bigquery.cloud.google.com/dataset/bigquery-public-data:the_met) ):

* gs://cloud-training-demos/images/met/APS6880.jpg
* gs://cloud-training-demos/images/met/DP205018.jpg
* gs://cloud-training-demos/images/met/DP290402.jpg
* gs://cloud-training-demos/images/met/DP700302.jpg

Use the Vision API to identify which of these images depict happy people and which ones depict unhappy people.

Hint (highlight to see): <p style="color:white">You will need to look for joyLikelihood and/or sorrowLikelihood from the response.</p>

In [0]:
import base64
IMAGE="gs://cloud-training-demos/images/met/APS6880.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses)

{'responses': [{}]}


In [0]:
import base64
IMAGE="gs://cloud-training-demos/images/met/DP205018.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses)

{'responses': [{'textAnnotations': [{'boundingPoly': {'vertices': [{'y': 3289, 'x': 1062}, {'y': 3289, 'x': 2290}, {'y': 3429, 'x': 2290}, {'y': 3429, 'x': 1062}]}, 'locale': 'en', 'description': 'AN EMBLE M OF A MERICA.\nLondon:Published Febla8, by Haines & Son,No,Rolls Buildings,Fe\n'}, {'boundingPoly': {'vertices': [{'y': 3289, 'x': 1062}, {'y': 3289, 'x': 1161}, {'y': 3359, 'x': 1161}, {'y': 3359, 'x': 1062}]}, 'description': 'AN'}, {'boundingPoly': {'vertices': [{'y': 3289, 'x': 1210}, {'y': 3289, 'x': 1534}, {'y': 3359, 'x': 1534}, {'y': 3359, 'x': 1210}]}, 'description': 'EMBLE'}, {'boundingPoly': {'vertices': [{'y': 3289, 'x': 1570}, {'y': 3289, 'x': 1630}, {'y': 3359, 'x': 1630}, {'y': 3359, 'x': 1570}]}, 'description': 'M'}, {'boundingPoly': {'vertices': [{'y': 3289, 'x': 1667}, {'y': 3289, 'x': 1762}, {'y': 3359, 'x': 1762}, {'y': 3359, 'x': 1667}]}, 'description': 'OF'}, {'boundingPoly': {'vertices': [{'y': 3289, 'x': 1810}, {'y': 3289, 'x': 1858}, {'y': 3359, 'x': 1858}, {

In [0]:
import base64
IMAGE="gs://cloud-training-demos/images/met/DP290402.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses)

{'responses': [{}]}


In [0]:
import base64
IMAGE="gs://cloud-training-demos/images/met/DP700302.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'gcs_image_uri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses)


{'responses': [{}]}


Copyright 2018 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.